# Module 3.1: Construct Dynamic Circuits    

**Task 3.1 from the IBM Qiskit v2.x Study Guide**. This notebook focuses on    
classical feedforward and control flow in circuits using Qiskit v2.x. It covers    
`if_test`, `if_else`, `for_loop`, and `while_loop`.

In [1]:

# Minimal imports and environment metadata
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import Gate
from qiskit.qasm3 import dumps as qasm3_dumps

# v2 primitives (do not create sessions; we will run locally with a fake backend)
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeTorino

import numpy as np
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

import sys
import qiskit as _qk
import qiskit_ibm_runtime as _rt

print(f"Python: {sys.version.split()[0]}")
print(f"Qiskit: {_qk.__version__}")
print(f"qiskit-ibm-runtime: {_rt.__version__}")

# Prepare a local fake backend for quick, deterministic runs
backend = FakeTorino()
sampler = SamplerV2(mode=backend)


Python: 3.12.9
Qiskit: 2.2.1
qiskit-ibm-runtime: 0.41.1


## 1) Conceptual Overview    

Dynamic circuits allow classical measurement results to influence later quantum    
operations in the same circuit execution. Control flow is defined at the circuit    
level and compiled to OpenQASM 3.    

**Key ideas**:    
- *Classical feedforward*: mid-circuit measurements update classical bits that    
  gate later actions.    
- *Control flow*: structured blocks expressed as    
  `QuantumCircuit.if_test`, `QuantumCircuit.if_else`,    
  `QuantumCircuit.for_loop`, and `QuantumCircuit.while_loop`.    
- *Semantics*: conditions evaluate classical bits or registers; bodies are    
  circuit fragments.    
- *Execution*: hardware that supports mid-circuit measurement executes these    
  natively; otherwise simulation interprets the control.    

We will keep examples small and reproducible. All randomness is controlled by    
setting seeds where relevant.

## 2) Hands-on Example (quick start)    

We build a 2-qubit, 1-bit circuit that measures `q0` and conditionally applies    
an `x` to `q1` when the measured bit `c0` is `1`. Then we sample with    
`SamplerV2` on a local fake backend.

In [2]:

# Quick start: dynamic if on one measured bit
qr = QuantumRegister(2, "q")
cr = ClassicalRegister(1, "c")
qc = QuantumCircuit(qr, cr, name="quick_if")

qc.h(qr[0])           # put q0 into |+>
qc.measure(qr[0], cr[0])

with qc.if_test((cr[0], 1)):
    qc.x(qr[1])       # flip q1 iff c0 == 1

qc.measure_all()

print(qc)
print("\nOpenQASM 3 preview:\n")
print(qasm3_dumps(qc))

# Run a small sampling job
job = sampler.run([qc], shots=1000)
res = job.result()

# Access counts from the first PUB
counts = res[0].data.meas.get_counts()
print("\nCounts:", dict(sorted(counts.items())))
plot_histogram(counts)
plt.show()


        ┌───┐┌─┐                           ░ ┌─┐   
   q_0: ┤ H ├┤M├───────────────────────────░─┤M├───
        └───┘└╥┘  ┌──────  ┌───┐ ───────┐  ░ └╥┘┌─┐
   q_1: ──────╫───┤ If-0  ─┤ X ├  End-0 ├──░──╫─┤M├
              ║   └──╥───  └───┘ ───────┘  ░  ║ └╥┘
              ║ ┌────╨────┐                   ║  ║ 
   c: 1/══════╩═╡ c_0=0x1 ╞═══════════════════╬══╬═
              0 └─────────┘                   ║  ║ 
meas: 2/══════════════════════════════════════╩══╩═
                                              0  1 

OpenQASM 3 preview:

OPENQASM 3.0;
include "stdgates.inc";
bit[1] c;
bit[2] meas;
qubit[2] q;
h q[0];
c[0] = measure q[0];
if (c[0]) {
  x q[1];
}
barrier q[0], q[1];
meas[0] = measure q[0];
meas[1] = measure q[1];



IBMInputValueError: 'The instruction h on qubits (0,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://quantum.cloud.ibm.com/docs/guides/transpile) for instructions to transform circuits and the primitive examples (https://quantum.cloud.ibm.com/docs/guides/primitives-examples) to see this coupled with operator transformations.'

## Sub-module A: `if_test` (context manager)    

`if_test` creates a conditional block that executes only when a condition on    
classical bits evaluates to true. Use it as a context manager; the optional    
`else_` handle captures the alternate path.    

**Docs**:    
https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow    

**Condition syntax**: a tuple `(clbit_or_creg, value)` or an expression on    
bits. The body is appended to the parent circuit in-place.

In [ ]:

# Example: single-bit condition, else branch using the context handle
qr = QuantumRegister(2, "q")
cr = ClassicalRegister(1, "c")
c = QuantumCircuit(qr, cr, name="if_test_demo")

c.h(qr[0])
c.measure(qr[0], cr[0])

with c.if_test((cr[0], 1)) as else_:
    c.x(qr[1])
with else_:
    c.h(qr[1])

c.measure(qr[1], 0)

print(c.draw())

job = sampler.run([c], shots=2000)
r = job.result()
counts = r[0].data.meas.get_counts()
print("Counts:", dict(sorted(counts.items())))
plot_histogram(counts)
plt.show()


## Sub-module B: `if_else` (structured alternative)    

`if_else` attaches two *circuits* as the true/false bodies. This is useful when    
you want to construct bodies independently and then splice them into a host    
circuit.    

**Docs**:    
https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow    

**Signature (simplified)**:    
$\texttt{QuantumCircuit.if\_else(condition, true\_body, false\_body,}\    
\texttt{ qubits=None, clbits=None)}$

In [ ]:

# Build bodies as separate circuits, then attach with if_else
qr = QuantumRegister(2, "q")
cr = ClassicalRegister(1, "c")
host = QuantumCircuit(qr, cr, name="if_else_demo")

true_body = QuantumCircuit(qr, cr, name="T")
true_body.x(qr[1])

false_body = QuantumCircuit(qr, cr, name="F")
false_body.h(qr[1])

host.h(qr[0])
host.measure(qr[0], cr[0])

host.if_else((cr[0], 1), true_body, false_body, qr, cr)

host.measure(qr[1], 0)

print(host.draw())

job = sampler.run([host], shots=2000)
r = job.result()
counts = r[0].data.meas.get_counts()
print("Counts:", dict(sorted(counts.items())))
plot_histogram(counts)
plt.show()


## Sub-module C: `for_loop` (static-count loop)    

`for_loop` repeats a body for a defined iteration set. The loop index can be    
bound to a *classical resource* for use inside the body, or ignored.    

**Docs**:    
https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow    

We implement a parity checker: apply `x` on `q1` once for each `1` seen while    
iterating over a 3-bit classical register that we fill via measurement.

In [ ]:

# For-loop that flips q1 for each measured 1 in a 3-bit classical register
qr = QuantumRegister(2, "q")
cr = ClassicalRegister(3, "c")
circ = QuantumCircuit(qr, cr, name="for_loop_demo")

# Prepare three biased bits by rotating and measuring q0 three times
for k, angle in enumerate([np.pi/8, np.pi/2, 3*np.pi/4]):
    circ.ry(angle, qr[0])
    circ.measure(qr[0], cr[k])
    circ.reset(qr[0])

# Loop over the 3 classical bits; flip q1 if the current bit equals 1
def body(loop, q, c):
    with loop.if_test((c[loop.index], 1)):
        loop.x(q[1])

circ.for_loop(range(3), None, body, qr, cr)

circ.measure(qr[1], 0)

print(circ.draw())

job = sampler.run([circ], shots=4000)
r = job.result()
counts = r[0].data.meas.get_counts()
print("Counts:", dict(sorted(counts.items())))
plot_histogram(counts)
plt.show()


## Sub-module D: `while_loop` (condition-controlled loop)    

`while_loop` repeats a body while a classical condition holds. Use with care:    
hardware imposes depth limits. For study purposes, we build a loop that keeps    
flipping a target qubit until a running flag bit becomes `0`.    

**Docs**:    
https://quantum.cloud.ibm.com/docs/en/guides/classical-feedforward-and-control-flow

In [ ]:

# While-loop that runs until a flag bit becomes 0
qr = QuantumRegister(2, "q")
cr = ClassicalRegister(1, "flag")
w = QuantumCircuit(qr, cr, name="while_loop_demo")

# Initialize flag=1 with high probability by measuring q0 from |+>
w.h(qr[0])
w.measure(qr[0], cr[0])

def body(loop, q, c):
    loop.x(q[1])          # act on data qubit
    # Re-randomize the flag from |+>, giving a 50% chance to exit next time
    loop.h(q[0])
    loop.measure(q[0], c[0])

w.while_loop((cr[0], 1), body, qr, cr)

w.measure(qr[1], 0)

print(w.draw())

job = sampler.run([w], shots=2000)
r = job.result()
counts = r[0].data.meas.get_counts()
print("Counts:", dict(sorted(counts.items())))
plot_histogram(counts)
plt.show()


## 6) Multiple Choice Questions    

**Q1.** Given the snippet below, which statement best describes the effect of the  
    
conditional block?    

```python
qc = QuantumCircuit(2, 1)
qc.h(0)
qc.measure(0, 0)
with qc.if_test((qc.clbits[0], 1)):
    qc.z(1)
qc.measure(1, 0)
```  

A. `z` on `q1` always applies, because `if_test` ignores measurements.    
B. `z` on `q1` applies only when the first measurement outcome is `1`.    
C. `z` on `q1` applies only when the first measurement outcome is `0`.    
D. The code is invalid; `if_test` cannot use a single classical bit.    

**Q2.** Which call correctly attaches two prepared bodies to a host circuit using  
    
`if_else`? (Assume `cond=(creg, 3)` and two-body circuits `t`, `f`.)    

A. `host.if_else(cond, t, f, host.qubits, host.clbits)`    
B. `host.if_else(t, f, cond, host.qubits, host.clbits)`    
C. `host.if_test(cond).else_(t, f)`    
D. `host.if_else(cond, [t, f])`

<details>
<summary><b>Answer Key</b></summary>  

- Q1: **B**. The `z` on `q1` executes when the measured bit equals 1.    
- Q2: **A**. `if_else` takes `(condition, true_body, false_body, qubits, clbits)`.  
    

</details>